<div style="width:1000 px">

<div style="float:right; width:98 px; height:98px;">
<img src="https://raw.githubusercontent.com/Unidata/MetPy/master/src/metpy/plots/_static/unidata_150x150.png" alt="Unidata Logo" style="height: 98px;">
</div>

<h1>MetPy Foundations</h1>
    <h3>MetPy for Quantitative Analysis of Meteorological Data</h3>

<div style="clear:both"></div>
</div>

<hr style="height:2px;">

### Topics
1. <a href="#units">Units and MetPy</a>
1. <a href="#constants">MetPy Constants</a>
1. <a href="#calculations">MetPy Calculations</a>

<a name="background"></a>
## Background
MetPy is a modern meteorological open-source toolkit for Python. It is a maintained project of [Unidata](http://www.unidata.ucar.edu) to serve the academic meteorological community. MetPy consists of three major areas of functionality:


### Plotting
As meteorologists, we have many field specific plots that we make. Some of these, such as the Skew-T Log-p require non-standard axes and are difficult to plot in most plotting software. In MetPy we've baked in a lot of this specialized functionality to help you get your plots made and get back to doing science. We will go over making different kinds of plots during the workshop.<br>
<center><img width="700" src="https://unidata.github.io/MetPy/latest/_images/sphx_glr_Station_Plot_001.png"/><br><i>Example of MetPy plotting tools</i></center>

### Calculations
Meteorology also has a common set of calculations that everyone ends up programming themselves. This is error-prone and a huge duplication of work. MetPy contains a set of well-tested calculations that is continually growing in an effort to be at feature parity with other legacy packages such as GEMPAK.
<center><a href="https://unidata.github.io/MetPy/latest/api/generated/metpy.calc.html" target="blank">MetPy Calculation Reference</a></center>

### File I/O
Finally, there are a number of odd file formats in the meteorological community. MetPy has incorporated a set of readers to help you deal with file formats that you may encounter during your research, including working with many xarray functions for data organization. 
<center><a href="https://unidata.github.io/MetPy/latest/api/generated/metpy.io.html" target="blank">MetPy I/O Reference</a> | <a href="https://unidata.github.io/MetPy/latest/api/generated/metpy.xarray.html" target="blank">MetPy xarray Reference</a></center>


<a name="units"></a>
## Units and MetPy

Early in our scientific careers we all learn about the importance of paying attention to units in our calculations. Unit conversions can still get the best of us and have caused more than one major technical disaster, including the crash and complete loss of the $327 million [Mars Climate Orbiter](https://en.wikipedia.org/wiki/Mars_Climate_Orbiter). 

MetPy uses the [pint](https://pint.readthedocs.io/en/latest/) library and a custom unit registry to help prevent unit mistakes in calculations. That means that every quantity you pass to MetPy should have units attached, just like if you were doing the calculation on paper. Attaching units is easy, simply multiply (`*`) the magnitude by the units in the format `units.___`.

In [1]:
# Import the MetPy unit registry
from metpy.units import units

In [2]:
length = 10.4 * units.inches
width = 20 * units.meters

print(length, width)

10.4 inch 20 meter


You can also use tab completion to see what units are available in the `units` registry.

Let's now attempt a rectangular area calculation with the above measurements. Multiplying length and width, we'll get an area in return with units attached.

In [3]:
area = length * width

print(area)

208.0 inch * meter


That's great, now we have an area, but it is not in a very useful unit...

MetPy can save you the headache of looking up conversions and maintaining high precision with the `.to()` method. You have the option of converting the individual measurements or the final area calculation. While you won't see m$^2$ in the units list, we can parse complex/compound units as strings:

In [4]:
area.to('m^2')

5.2832 <Unit('meter ** 2')>

<div class="alert alert-success">
    <b>POLL</b>: 

Which outputs the correct units for velocity?
</div>

`a = 10 * units.m / 20 * units.s`

`b = 10 * units.m / (20 * units.s)`

### Temperature
In meteorology, we frequently use three different measurements systems for temperature. We often get temperature in Kelvin from model output, but may want to report temperature for communication purposes in Celsius or Fahrenheit. To convert from one unit of temperature to another, we apply a conversion equation such as: $$F = (K − 273.15) * (9/5) + 32$$


These conversions are straightforward for simple one to one calculations. Where we run into trouble is when we refer to changes in temperature from one unit system to another. Temperature is a non-multiplicative unit - they are in a system with a reference point. That means that not only is there a scaling factor (9/5), but also an offset (32). This makes the math and unit book-keeping a little more complex. 

Imagine running a numerical model that tests the effect of surface temperature on cloud cover. Let's say we want to increase the baseline surface temperature of 290 K by 8 degrees Celsius. There may be many ways you can think of to complete this operation, so let's test a few methods:




Imagine adding 10 degrees Celsius to 100 degrees Celsius. Is the answer 110 degrees Celsius or 383.15 degrees Celsius (283.15 K + 373.15 K)? That's why there are delta degrees units in the unit registry for offset units. For more examples and explanation you can watch [MetPy Monday #13](https://www.youtube.com/watch?v=iveJCqxe3Z4).



In [5]:
# Starting simple, try adding 8 degrees C to 290 K
290 * units.kelvin + 8 * units.degC

OffsetUnitCalculusError: Ambiguous operation with offset unit (kelvin, degree_Celsius). See https://pint.readthedocs.io/en/stable/user/nonmult.html for guidance.

Notice that this fails with error `"Ambiguous operation with offset unit"` because we cannot add two units with offset reference points. 

Instead, we must look again at the problem we are trying to solve: Increase 290 K by 8 degrees Celsius. In this case, the 8 degrees Celsius is not a single temperature measurement, it is a representation of temperature change. On the Kelvin scale, we increase the starting temperature by an equivalent of 8 degrees Celsius, i.e. $\Delta$ 8 degrees Celsius.

MetPy (and pint) have a special unit to complete these kinds of calculations, `delta_degC`. Let's try our calculation again and find our resulting surface temperature:

In [6]:
290 * units.kelvin + 8 * units.delta_degC

298 <Unit('kelvin')>

<div class="alert alert-warning">
    <b>NOTE</b>
    
Absolute temperature scales like Kelvin and Rankine do not have an offset and therefore can be used in addition/subtraction without the need for a delta verion of the unit. For example, 
    
<code>>> 273 * units.kelvin + 10 * units.kelvin</code>

<code>>> 283 kelvin</code>
    
    
<code>>> 273 * units.kelvin - 10 * units.kelvin</code>

<code>>> 263 kelvin</code>
</div>

<a href="#top">Top</a>
<hr style="height:2px;">

<a name="constants"></a>
## MetPy Constants
Another common place that problems creep into scientific code is the value of constants. Can you reproduce someone else's computations from their paper? Probably not unless you know the value of all of their constants. Was the radius of the earth 6000 km, 6300km, 6371 km, or was it actually latitude dependent?

MetPy has a set of constants that can be easily accessed and make your calculations reproducible. You can view a [full table](https://unidata.github.io/MetPy/latest/api/generated/metpy.constants.html#module-metpy.constants) in the docs, look at the module docstring with `metpy.constants?` or checkout what's available with tab completion.

In [7]:
import metpy.constants as mpconst
import numpy as np

In [8]:
mpconst.earth_avg_radius

6371008.7714 <Unit('meter')>

In [9]:
mpconst.dry_air_molecular_weight

0.02896546 <Unit('kilogram / mole')>

You may also notice in the table that most constants have a short name as well that can be used:

In [10]:
mpconst.Re

6371008.7714 <Unit('meter')>

In [11]:
mpconst.Md

0.02896546 <Unit('kilogram / mole')>

In [12]:
mpconst.gamma_d

0.009761102563417645 <Unit('kelvin * kilogram * meter / joule / second ** 2')>

These constants can be plugged directly into equations to preserve precision. For example, as in the ideal gas law:
$$P = \rho R_dT_v$$

In [13]:
# Set density (rho) and virtual temperature (Tv) example values
rho = 1.18 * units('kg/m^3')
Tv = 296 * units.K

# Calculate pressure (P)
P = rho * mpconst.Rd * Tv

# Convert to hectopascal (hPa)
print(P.to(units.hPa))

1002.5994764851101 hectopascal


<a href="#top">Top</a>
<hr style="height:2px;">

<a name="calculations"></a>
## MetPy Calculations
MetPy also encompasses a set of calculations that are common in meteorology (with the goal of have all of the functionality of legacy software like GEMPAK and more). The [calculations documentation](https://unidata.github.io/MetPy/latest/api/generated/metpy.calc.html) has a complete list of the calculations in MetPy.

We'll scratch the surface and show off a few simple calculations here, but will be using many during the workshop.

In [14]:
import metpy.calc as mpcalc
import numpy as np

In [15]:
# Make some fake data for us to work with
np.random.seed(19990503)  # So we all have the same data
u = np.random.randint(0, 15, 10) * units('m/s')
v = np.random.randint(0, 15, 10) * units('m/s')

print(u)
print(v)

[14.0 2.0 12.0 5.0 3.0 5.0 14.0 8.0 9.0 10.0] meter / second
[6.0 10.0 7.0 11.0 10.0 13.0 2.0 3.0 5.0 0.0] meter / second


Let's use the `wind_direction` function from MetPy to calculate wind direction from these values. Remember you can look at the docstring or the website for help.

In [16]:
direction = mpcalc.wind_direction(u, v)
print(direction)

[246.80140948635182 191.30993247402023 239.74356283647072 204.44395478041653 196.69924423399362 201.03751102542182 261.86989764584405 249.44395478041653 240.94539590092285 270.0] degree


<div class="alert alert-success">
    <b>EXERCISE</b>: Calculate geostrophic wind speed
<br>
For a given unit of mass, the geostrophic wind is a balance between two forces: the Coriolis (CoF) and pressure gradient (PGF) forces
<br><br>
$$CoF = 2 \Omega V sin(\phi)$$
$$PGF = \frac{1}{\rho}\frac{\Delta P}{d}$$
<br>
    We can solve for the geostrophic wind speed of a unit mass using this equation:
$$V_g = \frac{1}{2 \Omega sin(\phi) \rho}\frac{\Delta P}{d}$$
<br>
    Calculate the geostrophic wind speed in units of $m/s$ for a unit mass with the following parameters: 
    $$P_1 = 500\,mb$$
    $$P_2 = 504\,mb$$
    $$d = 200\,km$$
    $$\phi = 40^{\circ}$$
    $$\rho = 0.70\,\frac{kg}{m^3}$$
</div>

In [17]:
deltaP = 4 * units.hPa
d = 200 * units.km
sinphi = np.sin(40 * units.degrees)
rho = 0.70 * units('kg/m^3')

In [18]:
vg = (1/(2*mpconst.omega*sinphi*rho))*(deltaP/d)
vg.to('m/s')

30.477613282299984 <Unit('meter / second')>

<div class="alert alert-success">
    <b>STRETCH EXERCISE</b>: Temperature change

A parcel at 60 degrees Fahrenheit is lifted dry adiabatically from the surface to a level of 1500 meters above ground level. 
    
Assuming a dry adiabatic lapse rate of -10 degrees C per 1000 meters, what is the final temperature of the parcel after lifting?
    
<b>Hint:</b> Remember to group your units and scalar magnitude with parentheses.
    
<b>Bonus:</b> Assuming a moist adiabatic lapse rate of -6 degrees C per 1000 meters, what is the temperature of the parcel if it continues lifting moist adiabatically an additional 2000 meters? (Final elevation of 3500 meters)

</div>

In [19]:
# define lapse rate
dalr = -10 * units.delta_degC / (1000 * units.meters)

# define starting temperature
parcel_t = 60 * units.degF

# lifting
parcel_t = parcel_t + dalr * (1500 * units.meters)
print("Parcel temp after dry adiabatic lift for 1500 m: " + str(parcel_t) + ", " + str(parcel_t.to('degC')))

# Bonus:
malr = -6 * units.delta_degC / (1000 * units.meters)
parcel_t = parcel_t + malr * (2000 * units.meters)
print("Parcel temp after moist adiabatic lift for additional 2000 m: " + str(parcel_t) + ", " + str(parcel_t.to('degC')))

Parcel temp after dry adiabatic lift for 1500 m: 33.0 degree_Fahrenheit, 0.5555555555555998 degree_Celsius
Parcel temp after moist adiabatic lift for additional 2000 m: 11.400000000000002 degree_Fahrenheit, -11.4444444444444 degree_Celsius


## More Information

### Further Practice
MetPy User Guide: https://unidata.github.io/MetPy/latest/userguide

MetPy Example Gallery: https://unidata.github.io/MetPy/latest/examples/index.html

### Save Your Work
<p>To save any of the files you modified or edited in this session:
    <ol>
        <li>Right click on any item in the left-side navigation pane</li>
        <li>Select <b>Download</b></li>
    </ol>
</p>

<p>
    To recreate the Conda environment used for this session on your local computer:
<ol>
	<li>
		Open a terminal (Linux or MacOS) or Anaconda Prompt (Windows). <br>
		Windows users: If Anaconda Prompt does not exist on your computer, Conda is not installed. Proceed with step 2.2.
	</li>
	<li>
		Confirm that Conda is installed by executing: <br>
		<b>conda --version</b>
		<ol>
			<li>
				If Conda is installed, a version number will be returned. Proceed to step 3.
			</li>
			<li>
				If Conda is not installed, proceed with the installation instructions provided for your operating system at <a href="https://unidata.github.io/python-training/#installing-conda" target="blank">this link</a>, then proceed to step 3.
			</li>
		</ol>
	</li>
	<li>
		Download the conda environment used in this workshop. On the link below, Shift + Right Click &gt; Save link as &gt; save the file as environment.yml in a location of your choosing.<br>
		<a href="https://raw.githubusercontent.com/Unidata/metpy-analysis/main/environment.yml">https://raw.githubusercontent.com/Unidata/metpy-analysis/main/environment.yml</a>
	</li>
	<li>
		In your terminal or command prompt, change directories to the location where the environment.yml file was saved. 
	</li>
	<li>
		Set up the course Python environment with the following command. <br>
		Note: this will take a few minutes to complete. <br>
		<b>conda env create -f environment.yml</b>
	</li>
	<li>
		Verify that the environment installed correctly by looking for <b>metpy-analysis</b> in your conda environment list <br>
		<b>conda env list</b>
	</li>
	<li>
		To use the new environment, activate the new environment<br>
		<b>conda activate metpy-analysis</b>
	</li>
	<li>
		Launch Jupyter Lab<br>
		<b>jupyter lab</b>
	</li>
</ol>
  
### Connect with Unidata
https://twitter.com/unidata

https://twitter.com/metpy

https://youtube.com/unidatanews

https://www.linkedin.com/company/unidatanews